# Test the endpoint

In [1]:
import json, os, sys
import pandas as pd
import numpy as np

In [2]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.pipeline_context import PipelineSession

from src.utils import config

In [4]:
client = boto3.client('sagemaker-runtime')

session = sagemaker.Session()
default_bucket = session.default_bucket()

iam = boto3.client('iam')
role_arn = iam.get_role(RoleName=f'101436505502-sagemaker-exec')['Role']['Arn']

# load local csv
df = pd.read_csv("data/mtsamples.csv")
df = df[df["transcription"].notna()]

## Run Inference on deployed endpoint

In [5]:
# get random sample
idx = np.random.randint(len(df))
input_txt = df.loc[idx].transcription
category = df.loc[idx].medical_specialty
print(category)
print(input_txt)

 ENT - Otolaryngology
PREOPERATIVE DIAGNOSES:,  OM, chronic, serous, simple or unspecified.  Adenoid hyperplasia.  Hypertrophy of tonsils.,POSTOPERATIVE DIAGNOSIS: , Same as preoperative diagnosis.,OPERATION: , Bilateral myringotomies with Armstrong grommet tubes, Adenoidectomy, and Tonsillectomy.,ANESTHESIA:,  General.,COMPLICATIONS:,  None.,ESTIMATED BLOOD LOSS: , Minimal.,DRAINS: , None.,CONSENT:,  The procedure, benefits, and risks were discussed in detail preoperatively.  The parentsagreed to proceed after all questions were answered.,TECHNIQUE: , The patient was brought to the operating room and placed in the supine position.  After general mask anesthesia was adequately obtained, the right external auditory canal was cleaned out under the microscope.  Serous fluid was aspirated from the middle ear space.  An Armstrong grommet tube was placed down through the incision and rotated into place.  The opposite ear was then cleaned out under the microscope.  Serous fluid was aspirated 

In [7]:
endpoint_name = "46-2023-04-11-08-57-35-184"
content_type = "text/csv" # "application/json" #
accept = "application/json" #"text/csv"
# payload = "test"#json.dumps({"instances":["test"]})

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    # CustomAttributes=custom_attributes,
    ContentType=content_type,
    Accept=accept,
    Body=input_txt,
)

json.loads(response["Body"].read())

{'prediction': [' Surgery']}

# Register model (for testing inference code)

In [14]:
gpu_instance_type = "ml.g4dn.xlarge"
pytorch_version = "1.9.0"
transformers_version = "4.11.0"
py_version = "py38"

requirement_dependencies = [os.path.join(
    # sys.path[0], 
    "images", "inference", "requirements.txt")]

model = HuggingFaceModel(
    name="text-classification-model-123",
    model_data="s3://sagemaker-eu-west-3-101436505502/model/pipelines-kam521dashky-train-model-JzSCBl76CF/output/model.tar.gz",
    sagemaker_session=session,
    source_dir="src",
    entry_point="model.py",
    dependencies=['requirements.txt'],
    role=role_arn,
    transformers_version=transformers_version,
    pytorch_version=pytorch_version,
    py_version=py_version,
)

## Run batch inference (Transformer)
Use latest approved model from model registry

In [88]:
def get_latest_approved_model(model_package_group_name="training-pipelineModelGroup"):
    sm_client = boto3.client('sagemaker')
    df = pd.DataFrame(sm_client.list_model_packages(
        ModelPackageGroupName=model_package_group_name)["ModelPackageSummaryList"])
    
    # models are allways sorted newst to oldest
    model_package_arn = df.loc[df.ModelApprovalStatus == "Approved"].iloc[0].ModelPackageArn
    
    # return model
    return sagemaker.ModelPackage(role=role_arn, model_package_arn=model_package_arn, sagemaker_session=session)

In [15]:
input_data_path = f"s3://{default_bucket}/data/test.csv"
output_data_path = f"s3://{default_bucket}/data/out"
CONTENT_TYPE_CSV = 'text/csv'

# load model
model = get_latest_approved_model(model_package_group_name)

# create transformer model
transformer =model.transformer(
    instance_count = 1,
    instance_type = 'ml.g4dn.xlarge', #"ml.m5.large",#
    # strategy = 'SingleRecord',
    # assemble_with = 'Line',
    output_path = output_data_path,
    accept = "application/json",
)

INFO:sagemaker:Creating model with name: text-classification-model-123


In [ ]:
transformer.transform(
    data=input_data_path,
    content_type=CONTENT_TYPE_CSV,
    # split_type='Line',
)
# transformer.wait()


# Torch test area

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm


class MyTokenizer:
    def __init__(self, model_name="distilbert-base-uncased") -> None:
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def tokenize(self, txt_input):
        return self.tokenizer.encode(txt_input, padding="max_length", truncation=True)
    
    
model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=40,
    )


class MyDataset(Dataset):
    def __init__(self, x, y) -> None:
        self.x = torch.tensor(x)
        self.y = torch.tensor(y)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [17]:
tok = MyTokenizer()
texts = list(df.transcription.values)

inputs = tok.tokenizer(texts, padding="max_length", return_tensors='pt', truncation=True)
inputs

{'input_ids': tensor([[  101, 20714,  1024,  ...,     0,     0,     0],
        [  101,  2627,  2966,  ...,  1010, 26572,   102],
        [  101,  2381,  1997,  ...,  4645,  2545,   102],
        ...,
        [  101, 20714,  1024,  ...,  8048,  1999,   102],
        [  101,  2708, 12087,  ..., 27179,  1024,   102],
        [  101,  2381,  1024,  ..., 17758,  2135,   102]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}

In [22]:
dataset = MyDataset(inputs.input_ids, inputs.attention_mask)

dataloader = DataLoader(dataset, shuffle=False, batch_size=10)


/var/folders/b8/mx2blhp93k7blwkppkkv1w_r0000gn/T/ipykernel_9997/2213946952.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x)
/var/folders/b8/mx2blhp93k7blwkppkkv1w_r0000gn/T/ipykernel_9997/2213946952.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y)


In [32]:
output = []
for (x, _) in tqdm(list(enumerate(dataloader))[:4]):
    outs = model(x)
    output += torch.argmax(outs.logits.cpu(), dim=1)

print(output)

  0%|          | 0/4 [00:00<?, ?it/s]


AttributeError: 'int' object has no attribute 'size'